In [ ]:
import numpy as np
from numpy import linalg as LA
from matplotlib import pyplot as plt
from scipy import ndimage
from util import Unit
from util import populate_plt_settings, get_column_width, get_fig_size, get_latex_float

from matplotlib import pyplot as plt
import matplotlib

In [ ]:
populate_plt_settings(plt)

In [ ]:
def plot_diagonal(filter_size, error_history, baseline_history):
    error_filtered = ndimage.uniform_filter(error_history,
                                            size=filter_size,
                                            mode='mirror')
    baseline_filtered = ndimage.uniform_filter(baseline_history,
                                               size=filter_size,
                                               mode='mirror')
    score_filtered = 1 - error_filtered / baseline_filtered
    
    num_rows = 1
    num_cols = 1
    matplotlib.rcParams['axes.unicode_minus'] = False
    fig, ax = plt.subplots(num_rows, num_cols, figsize = get_fig_size(get_column_width()))
    cmap = plt.get_cmap("tab10")

    piv_freq = 500
    ts = np.arange(len(score_filtered)) / piv_freq
    ax.plot(ts, score_filtered)
    ax.set_xlabel(r'$t$ (s)')
    ax.set_ylim(0, 1)
    ax.set_ylabel(r"2D Eulerian score")
    ax.set_xlim(0, 20)

    fig.tight_layout(pad=0.05) # pad is 1.08 by default https://stackoverflow.com/a/59252633

    ax.axvspan(0.833, 4.85, color=cmap(1), alpha=0.5)
    ax.axvspan(5.083, 9.1, color=cmap(3), alpha=0.5)
    ax.annotate("Diagonal 1", xy=(2.7, 0.9), xycoords="data",
                      va="center", ha="center",
                      bbox=dict(boxstyle="square,pad=0.3", fc="w", ec="black", lw=1.5, alpha=0.5))

    ax.annotate("Diagonal 2", xy=(7.4, 0.9), xycoords="data",
                      va="center", ha="center",
                      bbox=dict(boxstyle="square,pad=0.3", fc="w", ec="black", lw=1.5, alpha=0.5))
    
    epsilon = 0.01
    ax.xaxis.set_ticks(np.arange(0, 20 + epsilon, 1))
    ax.yaxis.set_ticks(np.arange(0, 1 + epsilon, 0.1))
    fig.savefig('linear-stir-two-diagonals-time-score.pgf', bbox_inches='tight') # bbox_inches='tight' necessary for keeping the time legend inside the canvas

In [ ]:
metric = 'eulerian_masked'
recon_dir = '/home/kennychufk/workspace/pythonWs/alluvion-optim/val-0416_103739-2v7m4muc-2600-16813553'
error_history = np.load(f'{recon_dir}/{metric}_error.npy')
baseline_history = np.load(f'{recon_dir}/{metric}_baseline.npy')
num_samples_history = np.load(f'{recon_dir}/{metric}_num_samples.npy')

filter_size = 200
plot_diagonal(filter_size, error_history, baseline_history)

In [ ]:
def plot_linear_circular(filter_size, error_history, baseline_history):
    error_filtered = ndimage.uniform_filter(error_history,
                                            size=filter_size,
                                            mode='mirror')
    baseline_filtered = ndimage.uniform_filter(baseline_history,
                                               size=filter_size,
                                               mode='mirror')
    score_filtered = 1 - error_filtered / baseline_filtered
    
    num_rows = 1
    num_cols = 1
    matplotlib.rcParams['axes.unicode_minus'] = False
    fig, ax = plt.subplots(num_rows, num_cols, figsize = get_fig_size(get_column_width()))
    cmap = plt.get_cmap("tab10")

    piv_freq = 500
    ts = np.arange(len(score_filtered)) / piv_freq
    ax.plot(ts, score_filtered)
    ax.set_xlabel(r'$t$ (s)')
    ax.set_ylim(0, 1)
    ax.set_ylabel(r"2D Eulerian score")
    ax.set_xlim(0, 14)

    fig.tight_layout(pad=0.05) # pad is 1.08 by default https://stackoverflow.com/a/59252633

    ax.axvspan(0.983, 6.78, color=cmap(1), alpha=0.5)
    ax.axvspan(7.15, 10.9, color=cmap(3), alpha=0.5)
    ax.annotate("Linear", xy=(4.0, 0.9), xycoords="data",
                      va="center", ha="center",
                      bbox=dict(boxstyle="square,pad=0.3", fc="w", ec="black", lw=1.5, alpha=0.5))

    ax.annotate("Circular", xy=(9.1, 0.9), xycoords="data",
                      va="center", ha="center",
                      bbox=dict(boxstyle="square,pad=0.3", fc="w", ec="black", lw=1.5, alpha=0.5))
    epsilon = 0.01
    ax.xaxis.set_ticks(np.arange(0, 20 + epsilon, 1))
    ax.yaxis.set_ticks(np.arange(0, 1 + epsilon, 0.1))
    fig.savefig('linear-circular-stir-time-score.pgf', bbox_inches='tight') # bbox_inches='tight' necessary for keeping the time legend inside the canvas

In [ ]:
metric = 'eulerian_masked'
recon_dir = '/home/kennychufk/workspace/pythonWs/alluvion-optim/val-0416_114327-2v7m4muc-2600-16813553'
error_history = np.load(f'{recon_dir}/{metric}_error.npy')
baseline_history = np.load(f'{recon_dir}/{metric}_baseline.npy')
num_samples_history = np.load(f'{recon_dir}/{metric}_num_samples.npy')

filter_size = 200
plot_linear_circular(filter_size, error_history, baseline_history)